In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pickle
import numpy
import argparse
import os

from utils import *
from autoencoder import *

torch.set_float32_matmul_precision('medium')

In [2]:
top = md.load_prmtop('/orange/alberto.perezant/t.desilva/insulin/water/no_water_aligned/system.parm7')
comp = '_compressed.pkl'
model = 'model.pt'

In [3]:
# Define device ---------
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Device name:', torch.cuda.get_device_name(device))
else:
    device = torch.device('cpu')
    print('Device name: CPU')

Device name: NVIDIA A100-SXM4-80GB


In [4]:
decoder = torch.load(model).to(device).model.decoder
comp = torch.concatenate(pickle.load(open(comp, 'rb')))

In [5]:
# with torch.no_grad():
#     decoder.eval()
#     np_traj = decoder(comp).detach().cpu().numpy()
#     np_traj = np_traj.reshape(-1,np_traj.shape[2],3)

In [6]:
# # md.Trajectory(numpy_traj, topology).save_xtc('output_trajectory.xtc')
# md.Trajectory(np_traj, top).save_netcdf('output_trajectory.nc')

In [25]:
with torch.no_grad():
    decoder.eval()
    # Open the NetCDF file for writing in append mode
    with md.formats.netcdf.NetCDFTrajectoryFile('output_trajectory.nc', 'w') as f:
        for i in range(len(comp)):  # Iterate over your input data
            np_traj_frame = decoder(comp[i].reshape(1,-1)).detach().cpu().numpy()
            np_traj_frame = np_traj_frame.reshape(-1, np_traj_frame.shape[2], 3)
            
            f.write(np_traj_frame)  # Write single frame